In [ ]:
!pip install diffusers

In [ ]:
from google.colab import files
uploaded = files.upload()

In [16]:
# @title Default title text
# Choose uploaded filename
filename = list(uploaded.keys())[0]

# Choose image size option
size_option = "square"  # @param ["portrait", "square" ,"landscape"]


# Define dimensions
size_map = {
    "portrait": (512, 768),
    "square": (512, 512),
    "landscape": (768, 512)
}
width, height = size_map[size_option]

In [ ]:
!curl -L -o meinamix12-sd-1.5.safetensors -H "Authorization: Bearer civit_ai_token" "https://civitai.com/api/download/models/948574?type=Model&format=SafeTensor&size=pruned&fp=fp16"

In [ ]:
model = "meinamix12-sd-1.5.safetensors"

In [ ]:
checkpoint_repo = "finnstrom3693/sd-turbo"

In [4]:
checkpoint_repo = "nqzfaizal77ai/austersight-exp-sd-1.5"

In [ ]:
checkpoint_repo = "finnstrom3693/meinamix-sd-1.5-v12"

In [5]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
from PIL import Image

In [6]:
precision = 'half' # @param ["half", "full" ,"bf16"]
if precision == "half":
    selected_precision = torch.float16
elif precision == "full":
    selected_precision = torch.float32
elif precision == "bf16":
    selected_precision = torch.bfloat16

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# Load pipeline
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    checkpoint_repo,
    torch_dtype=selected_precision,
).to(device)

In [ ]:
# # Load from single file
# pipe = StableDiffusionImg2ImgPipeline.from_single_file(
#     model,
#     torch_dtype=selected_precision,
# ).to(device)

In [9]:
pipe.safety_checker = None
pipe.requires_safety_checker = False
# DPMSolverMultistepScheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(
    pipe.scheduler.config,
    algorithm_type="sde-dpmsolver++",
    use_karras_sigmas=True
)

In [17]:
# Load and resize input image
init_image = Image.open(filename).convert("RGB")
init_image = init_image.resize((width, height), Image.LANCZOS)

In [1]:
prompt = "sidebangs hairstyle with earring, sibear, 1girl, solo, library, bookshelf, books, sitting, chair, indoors, looking_at_viewer, long_hair, sidebangs, brown_hair, glasses, adjusting_glasses, necklace, black_choker, lace_top, black_skirt, thigh_straps, cleavage, confident_pose, seductive_pose, detailed_background, elegant, gothic_fashion, jewelry, realistic, anime_style"  # @param {type:"string"}
negative_prompt = "lowres,glasses,text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature"  # @param {type:"string"}
num_images = 4  # @param {type:"integer"}
strength = 0.5 # @param {type:"slider", min:0, max:1, step:0.05}
guidance_scale = 9 # @param {type:"slider", min:0, max:15, step:0.5}

In [ ]:
# Generate images
generated_images = pipe(
    prompt=[prompt] * num_images,
    negative_prompt=[negative_prompt] * num_images,
    image=[init_image] * num_images,
    strength=strength,
    guidance_scale=guidance_scale,
).images

In [20]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_images(images):
    N = len(images)
    n_cols = 2
    n_rows = int(np.ceil(N / n_cols))

    plt.figure(figsize = (20, 5 * n_rows))
    for i in range(len(images)):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.title(i)
        plt.imshow(np.array(images[i]))
        plt.axis(False)
    plt.show()
plot_images(generated_images)

In [22]:
for i, img in enumerate(generated_images):
    filename = f"output_image_{i+1}.png"
    img.save(filename)